# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [3]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

## Obtenir la liste des politiciens belges

In [49]:
def execute_query(sparql, statement, retries=3, delay=5):
    for attempt in range(retries):
        try:
            sparql.setQuery(statement)
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            return results
        except Exception as e:
            print(f"Error: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2  # Increase the delay for each retry

# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = country of citizenship 
# Q31 = Belgium
# P106 = Opccupation
# Q82955 = Politician
# P569 = date of birth
# P570 = date of death
statement = """
SELECT DISTINCT ?journalist ?journalistLabel ?countryLabel ?family_nameLabel ?given_nameLabel ?date_of_birth ?date_of_death ?place_of_birthLabel ?place_of_deathLabel ?isni ?NTA ?BnF ?genderLabel ?native_languageLabel
WHERE {
  ?journalist p:P106 ?statement.
  ?statement ps:P106 wd:Q1930187.
  OPTIONAL {?journalist wdt:P734 ?family_name.}
  OPTIONAL {?journalist wdt:P735 ?given_name.}
  OPTIONAL {?journalist wdt:P569 ?date_of_birth.}
  OPTIONAL {?journalist wdt:P570 ?date_of_death.}
  OPTIONAL {?journalist wdt:P19 ?place_of_birth.}
  OPTIONAL {?journalist wdt:P20 ?place_of_death.}
  OPTIONAL {?journalist wdt:P213 ?isni.}
  OPTIONAL {?journalist wdt:P1006 ?NTA.}
  OPTIONAL {?journalist wdt:P268 ?BnF.}
  OPTIONAL {?journalist wdt:P21 ?gender.}
  OPTIONAL {?journalist wdt:P103 ?native_language.}

  ?journalist wdt:P27 ?nationality.
  
  FILTER (?nationality = wd:Q974 || ?nationality = wd:Q31) #Filtre congolaise ou belge
  
 
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr,en,nl". }
}
ORDER BY ?journalistLabel


"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Belgian Journalist found\n")
print(rows[:10])


1538 Belgian Journalist found

[{'journalist': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q110110934'}, 'journalistLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Alphonse Elleboudt'}, 'family_nameLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Elleboudt'}, 'given_nameLabel': {'xml:lang': 'fr', 'type': 'literal', 'value': 'Alphonse'}, 'date_of_birth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1864-02-06T00:00:00Z'}, 'date_of_death': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1948-06-08T00:00:00Z'}, 'place_of_birthLabel': {'xml:lang': 'fr', 'type': 'literal', 'value': 'Ostende'}, 'place_of_deathLabel': {'xml:lang': 'fr', 'type': 'literal', 'value': 'Ostende'}, 'genderLabel': {'xml:lang': 'fr', 'type': 'literal', 'value': 'masculin'}}, {'journalist': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q9154659'}, 'journalistLabel': {'xml:lang': 'en', 'type': 'liter

## Filtrer pour n'afficher que les noms contenant "Alphonse"

In [54]:
genderLabel = 'female'
max_results = 10

date_format = "%Y-%m-%dT%H:%M:%SZ"
filtered_rows = [row for row in rows if genderLabel in row['personLabel']['value']]
print(f"Displaying the first {max_results}:\n")
for row in filtered_rows[:max_results]:
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format)
        birth_year = birth_date.year
    except ValueError:
        birth_year = "????"
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format)
        death_year = death_date.year
    except ValueError: # unknown death date
        death_year = "????"
    except KeyError: # still alive
        death_year = ""
    print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

KeyError: 'personLabel'

## Pour en savoir plus

- [Le projet Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)
- [Aide à la construction de requêtes](https://query.wikidata.org/)